In [0]:
# Trip Data
from pyspark.sql.types import StructType, StructField, StringType

trip_schema = StructType([
  StructField("ride_id", StringType(), True),
  StructField("rideable_type", StringType(), True),
  StructField("started_at", StringType(), True),
  StructField("ended_at", StringType(), True),
  StructField("start_station_name", StringType(), True),
  StructField("start_station_id", StringType(), True),
  StructField("end_station_name", StringType(), True),
  StructField("end_station_id", StringType(), True),
  StructField("start_lat", StringType(), True),
  StructField("start_lng", StringType(), True),
  StructField("end_lat", StringType(), True),
  StructField("end_lng", StringType(), True),
  StructField("member_casual", StringType(), True)
])

(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("header", "true")
  .option("cloudFiles.schemaLocation", "dbfs:/mnt//trips/trip_data/schema")
  .option("cloudFiles.rescuedDataColumn", "_rescued_data")
  .option("cloudFiles.schemaEvolutionMode", "rescue")         
  .schema(trip_schema)
  .load("abfss://divvycontainer@divvystorage1.dfs.core.windows.net/trips") 
  .writeStream
  .option("checkpointLocation", "dbfs:/mnt/trips/trip_data/data")
  .trigger(availableNow=True)
  .toTable("bronze_trip_data")
).awaitTermination()

In [0]:
# Weather Data
from pyspark.sql.types import StructType, StructField, StringType

weather_schema = StructType([
  StructField("YEAR", StringType(), True),
  StructField("MO", StringType(), True),
  StructField("DY", StringType(), True),
  StructField("HR", StringType(), True),
  StructField("TEMP", StringType(), True),
  StructField("PRCP", StringType(), True),
  StructField("HMDT", StringType(), True),
  StructField("WND_SPD", StringType(), True),
  StructField("ATM_PRESS", StringType(), True),
  StructField("REF", StringType(), True)
])

(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("header", "true") 
  .option("cloudFiles.schemaLocation", "dbfs:/mnt/weather/weather_data/schema")
  .option("cloudFiles.rescuedDataColumn", "_rescued_data")
  .option("cloudFiles.schemaEvolutionMode", "rescue")
  .schema(weather_schema)
  .load("abfss://divvycontainer@divvystorage1.dfs.core.windows.net/trips")
  .writeStream
  .option("checkpointLocation", "dbfs:/mnt/weather_data/data")
  .trigger(availableNow=True)
  .toTable("bronze_weather_data")
).awaitTermination()